<a href="https://colab.research.google.com/github/joje3029/AI_Study/blob/main/2023_08_17_%EC%97%90%EC%96%B4%EB%B9%84%EC%97%94%EB%B9%84_2%EC%B0%A8_%EB%8F%84%EC%A0%84!_1%EC%B0%A8%20%EC%9E%98%EB%AA%BB%EB%90%9C%EA%B1%B0_%EA%B3%A0%EC%B3%90%EC%84%9C_%EC%A0%90%EC%88%98_%EC%98%AC%EB%A6%AC%EC%9E%90!_%EC%84%B1%EA%B3%B5!!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [3]:
# 데이터 설명
# id: 일련번호
# name: AirBnB 이름
# host_id: 호스트 ID
# host_name: 호스트 성명
# neighbourhood_group: 지구명
# neighbourhood : 지역명
# latitude: 위도
# longitude: 경도
# room_type: 방 타입
# price: 가격
# minimum_nights: 최소 숙박일수
# number_of_reviews: 리뷰 개수
# last_review: 최근 리뷰 일자
# reviews_per_month: 월 평균 리뷰 수
# calculated_host_listings_count:  호스트에게 대여 가능한 에어비앤비의 총 개수
# availability_365: 1년 중 가능한 대여일

# 에어비엔비 가격 예상이 목표

In [7]:
# 1. 데이터 로드
AB=pd.read_csv('/content/AB_NYC_2019.csv')
# AB:id(int),	name(object)(null),	host_id(int),	host_name(object)(null),	neighbourhood_group(object),	neighbourhood(object),	latitude(float),	longitude(float),	room_type(object),	price(int),	minimum_nights(int),	number_of_reviews(int),	last_review(object)(null),	reviews_per_month(float),	calculated_host_listings_count(int),	availability_365(int)
AB.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [4]:
# 4. 불필요해 보이는 컬럼 삭제
# (1)컬럼중에서 중복되는거나 가격 예측에 상관 없어보이는 칼럼이 뭔지 파악
# id: 일련번호 -> 그냥 구분하기 위한 일련 번호 : 가격 예측과 관계 없어 보임 * 없애기!
# name: AirBnB 이름 -> 가격 예측과 관계 없어 보임 * 없애기!
# host_id: 호스트 ID
# host_name: 호스트 성명-> 호스트 아이디와 중복 * 없애기!
# neighbourhood_group: 지구명
# neighbourhood : 지역명 -> 지구명과 중복 * 없애기!
# latitude: 위도-> 지구명과 중복 * 없애기!
# longitude: 경도-> 지구명과 중복 * 없애기!
# room_type: 방 타입
# price: 가격-> 정답
# minimum_nights: 최소 숙박일수
# number_of_reviews: 리뷰 개수
# last_review: 최근 리뷰 일자-> 가격 예측과 관계 없어 보임 * 없애기!
# reviews_per_month: 월 평균 리뷰 수-> 가격 예측과 관계 없어 보임 * 없애기!
# calculated_host_listings_count:  호스트에게 대여 가능한 에어비앤비의 총 개수-> 가격 예측과 관계 없어 보임 * 없애기!
# availability_365: 1년 중 가능한 대여일-> 가격 예측과 관계 없어 보임 * 없애기!


In [15]:
# (2)불필요해 하다고 판단판 컬럼 삭제하기
modi_AB=AB.drop(columns=['id','name','host_name','neighbourhood','latitude','longitude','last_review','reviews_per_month','calculated_host_listings_count','availability_365'])
# modi_AB : host_id(int), neighbourhood_group(object), room_type(object), price(int) , minimum_nights(int), number_of_reviews(int) -> 결측치 없음.
# modi_AB.describe() : 이상치가 존재하는거 같음. -> IQR 하고 나서 이상치 제거 해야함.
modi_AB.head()

,host_id,neighbourhood_group,room_type,price,minimum_nights,number_of_reviews
0,2787,Brooklyn,Private room,149,1,9
1,2845,Manhattan,Entire home/apt,225,1,45
2,4632,Manhattan,Private room,150,3,0
3,4869,Brooklyn,Entire home/apt,89,1,270
4,7192,Manhattan,Entire home/apt,80,10,9


In [19]:
# 5.데이터 클리닝
## 이상치 제거
# IQR : Q3(3분기)-Q1(1분기)
Q1=modi_AB.quantile(0.25) #1분기
Q3=modi_AB.quantile(0.75) #3분기

IQR=Q3-Q1 #IQR 계산

highest = Q3+1.5*IQR #상한치
lowest = Q1-1.5*IQR #하한치



<ipython-input-19-d3e05eff0497>:4: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q1=modi_AB.quantile(0.25) #1분기
<ipython-input-19-d3e05eff0497>:5: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q3=modi_AB.quantile(0.75) #3분기


In [ ]:
# modi_AB 데이터 중에 상한치 이상과 하한치 이하인(이상치인) 데이터값 제외

outline_range=(modi_AB>highest)|(modi_AB<lowest) # flase(0) : 이상치 아님. True(1)는 이상치임

def outlineDrop(row):
  if row.sum() >0: #이상치
    return False
  else:
    return True

notoutline_AB=outline_range.apply(outlineDrop, axis=1)
normal_AB=modi_AB.loc[notoutline_AB]
normal_AB

In [43]:
# 6. 데이터 전처리
## 범주형 데이터 ont-hot 인코딩
## 수치형 데이터 표준화

# (1)범주형 데이터(neighbourhood_group, room_type )와 수치형 데이터(host_id , minimum_nights , number_of_reviews ), 정답데이터(price) 를 분리
object_AB = normal_AB[['neighbourhood_group', 'room_type']]
object_AB=object_AB.reset_index() # 범주형 데이터

num_AB = normal_AB[['host_id' , 'minimum_nights' , 'number_of_reviews']]
num_AB=num_AB.reset_index() #수치형 데이터

answer_AB = normal_AB[['price']]
answer_AB=answer_AB.reset_index() # 정답 데이터 32863

In [ ]:
# (2)범주형 데이터 one-hot 인코딩
encoded_AB=pd.get_dummies(object_AB) #32863

In [47]:
# (3)수치형 데이터 표준화
from sklearn.preprocessing import StandardScaler

standard = StandardScaler()

scaled_AB=standard.fit_transform(num_AB)
scaled_AB=pd.DataFrame(scaled_AB, columns=num_AB.columns)


In [57]:
# (4)concat 범주형과 수치형
new_AB=pd.concat([encoded_AB, scaled_AB], axis=1)
train=new_AB
target=answer_AB

In [58]:
# 7. 훈련데이터와 타겟데이터 구분하고 훈련용, 테스트용 분리하기
from sklearn.model_selection import train_test_split

trd, tsd, trt, tst = train_test_split(train ,target, test_size=0.8, random_state=42)


In [61]:
# 8. 선형회귀 모델 학습 및 평가
# (1)학습
from sklearn.linear_model import LinearRegression

Linear = LinearRegression()

Linear.fit(trd, trt)


LinearRegression()

In [63]:
# (2) 평가 : 피처같은거 안하고 결과
# Linear.score(trd,trt) : 0.7252076686511453
# Linear.score(tsd,tst) : 0.7175439063272717

0.7252076686511453